# Restaurant Consolidator — B2C Portal Revamp (Intelligent Automation)
**Notebook created:** 2025-11-05 17:42:32
 
**Goal:** Provide a complete end-to-end exploratory data analysis, cleaning, and feature inspection workflow to identify *star restaurants* and generate recommendations. This notebook is produced as a deliverable for the uploaded dataset (zip file) and includes code, explanations, and visualizations. Use this notebook as the base for modeling or Tableau dashboard creation.

**Files expected (from provided zip):**
- `data` — main dataset with 19 attributes
- `Country-Code` — country code mapping (2 attributes)

The notebook performs:
1. Importing, understanding and inspecting data (structure, missing values, duplicates)
2. Cleaning (duplicate removal, basic imputations)
3. EDA: geography, franchising, table booking, delivery, votes
4. EDA: cuisines, cost distribution, ratings, factors affecting ratings
5. Visualizations (matplotlib) and suggestions for Tableau dashboarding
6. Next steps: recommend metrics and simple recipe for building a recommendation engine

**How to run:** Execute all cells in order. The notebook attempts to locate the zip file at `/mnt/data/1683266696_dataset (2).zip` — change the path if needed.


In [ ]:

# Unzip the provided dataset and list files
import zipfile, os, sys, pathlib
zip_path = r"/mnt/data/1683266696_dataset (2).zip"

if not os.path.exists(zip_path):
    print("Zip file not found at:", zip_path)
    print("Please upload the dataset zip to /mnt/data or update the path.")
else:
    extract_to = "/mnt/data/restaurant_dataset_extracted"
    os.makedirs(extract_to, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_to)
    print("Extracted to:", extract_to)
    for root, dirs, files in os.walk(extract_to):
        for f in files:
            print(os.path.join(root,f))


In [ ]:

# Standard imports and display settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import os, textwrap

pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 160)


In [ ]:

# Attempt to locate main data files inside extracted folder
base = "/mnt/data/restaurant_dataset_extracted"
candidates = []
for root, dirs, files in os.walk(base):
    for f in files:
        if f.lower().endswith(('.csv', '.txt', '.xls', '.xlsx')):
            candidates.append(os.path.join(root,f))
candidates


In [ ]:

# Heuristic loader: choose likely files by name
candidates = _  # from previous cell's output
data_path = None
country_code_path = None

for p in candidates:
    name = os.path.basename(p).lower()
    if 'country' in name or 'country-code' in name or 'country code' in name:
        country_code_path = p
    if 'data' == os.path.splitext(name)[0] or 'data.csv' in name or 'zomato' in name or 'restaurant' in name:
        data_path = p

# fallback: if only two CSVs present, assign them
if not data_path and len(candidates)>=1:
    data_path = candidates[0]
if not country_code_path and len(candidates)>=2:
    country_code_path = candidates[1]

print("Data file:", data_path)
print("Country code file:", country_code_path)

# Load files if found
data = pd.read_csv(data_path, encoding='latin1') if data_path else None
country_code = pd.read_csv(country_code_path, encoding='latin1') if country_code_path else None

print("Data shape:", None if data is None else data.shape)
print("Country code shape:", None if country_code is None else country_code.shape)

# show top rows
display(data.head())
display(country_code.head())


In [ ]:

# 1. Preliminary data inspection
def inspect_df(df, name="data"):
    display(Markdown(f"### Inspecting `{name}`"))
    print("Shape:", df.shape)
    print("\nColumn dtypes:")
    print(df.dtypes)
    print("\nMissing values (per column):")
    print(df.isnull().sum().sort_values(ascending=False).head(20))
    print("\nDuplicate rows:", df.duplicated().sum())
    print("\nSample rows:")
    display(df.sample(5, random_state=42))

inspect_df(data, "data")
inspect_df(country_code, "country_code")


In [ ]:

# 2. Identify duplicates and remove them (from `data`)
initial_shape = data.shape
dupe_count = data.duplicated().sum()
print("Initial shape:", initial_shape)
print("Duplicate rows found:", dupe_count)

# Inspect duplicates (example)
if dupe_count>0:
    display(data[data.duplicated(keep=False)].head(10))

# Drop exact duplicate rows
data_clean = data.drop_duplicates().copy()
print("Shape after removing exact duplicates:", data_clean.shape)

# If there are duplicates based on subset keys (like Restaurant ID, Name + Address), attempt to drop
# Heuristic: common keys 'Restaurant ID', 'Restaurant_ID', 'Restaurant ID', 'url' might exist
possible_id_cols = [c for c in data_clean.columns if any(x in c.lower() for x in ['id','restaurant id','res_id','res id','name'])]
possible_id_cols[:10]


In [ ]:

# 3. Geographical distribution: cities with max and min restaurants
# We'll attempt to find likely city and country columns
cols = [c.lower() for c in data_clean.columns]
city_col = None
country_col = None
for c in data_clean.columns:
    if 'city' in c.lower():
        city_col = c
    if 'country' in c.lower():
        country_col = c

print("Detected city column:", city_col)
print("Detected country column:", country_col)

if city_col:
    city_counts = data_clean[city_col].value_counts(dropna=False)
    display(city_counts.head(10))
    max_city = city_counts.idxmax()
    min_city = city_counts[city_counts>0].idxmin()
    print(f"City with maximum restaurants: {max_city} ({city_counts.max()})")
    print(f"City with minimum restaurants (non-zero): {min_city} ({city_counts[city_counts>0].min()})")

# 4. Franchise with most national presence
# Find a column likely to indicate 'is franchise' or 'has franchise' or 'chain'
chain_col = None
for c in data_clean.columns:
    if any(k in c.lower() for k in ['chain','franchise','establishment','brand']):
        chain_col = c
        break
print("Detected chain/franchise-like column:", chain_col)

# If dataset has 'Restaurant Name' we can count occurrences
name_col = None
for c in data_clean.columns:
    if 'name' in c.lower():
        name_col = c
        break

if name_col and city_col:
    franchise_counts = data_clean.groupby(name_col)[city_col].nunique().sort_values(ascending=False)
    display(franchise_counts.head(10))
    print("Franchise with most national presence (based on unique cities):")
    display(franchise_counts.head(5))


In [ ]:

# 5. Table booking ratio
tb_col = None
for c in data_clean.columns:
    if 'table' in c.lower() and 'book' in c.lower():
        tb_col = c
        break
print("Detected table booking column:", tb_col)

if tb_col:
    tb_counts = data_clean[tb_col].value_counts(dropna=False)
    display(tb_counts)
    tb_yes = tb_counts.get('Yes', tb_counts.get('yes', tb_counts.get(1,0)))
    tb_no = tb_counts.get('No', tb_counts.get('no', tb_counts.get(0,0)))
    total = tb_counts.sum()
    print(f"Table booking ratio: Yes {tb_yes}/{total} = {tb_yes/total:.2%}, No {tb_no}/{total} = {tb_no/total:.2%}")

# 6. Percentage of restaurants providing online delivery
od_col = None
for c in data_clean.columns:
    if 'online' in c.lower() and 'delivery' in c.lower() or c.lower()=='online_delivery' or 'deliv' in c.lower() and 'online' in c.lower():
        od_col = c
        break
# fallback search for 'delivery' column
if not od_col:
    for c in data_clean.columns:
        if 'delivery' in c.lower():
            od_col = c
            break

print("Detected delivery column:", od_col)
if od_col:
    od_counts = data_clean[od_col].value_counts(dropna=False)
    display(od_counts.head(10))
    # Try parse yes/no or 1/0
    online_yes = od_counts.get('Yes', od_counts.get('yes', od_counts.get(1,0)))
    total = od_counts.sum()
    print(f"Online delivery percentage: {online_yes}/{total} = {online_yes/total:.2%}")

# 7. Difference in number of votes for restaurants that deliver vs those that do not deliver
votes_col = None
for c in data_clean.columns:
    if 'vote' in c.lower() or 'votes' in c.lower() or 'votes_count' in c.lower():
        votes_col = c
        break
print("Detected votes column:", votes_col)
if votes_col and od_col:
    # convert votes to numeric if needed
    data_clean['__votes_num__'] = pd.to_numeric(data_clean[votes_col], errors='coerce')
    grp = data_clean.groupby(data_clean[od_col].astype(str))['__votes_num__'].agg(['count','mean','sum','median']).sort_values('mean', ascending=False)
    display(grp)
    # difference in total votes between deliverers and non-deliverers (if values present)
    vals = grp['sum']
    if len(vals)>=2:
        print("Difference in total votes between top two delivery groups:", vals.iloc[0] - vals.iloc[1])


In [ ]:

# 1. Top 10 cuisines across cities (or overall)
# Find cuisine column
cuisine_col = None
for c in data_clean.columns:
    if 'cuisine' in c.lower() or 'cuisines' in c.lower():
        cuisine_col = c
        break
print("Detected cuisine column:", cuisine_col)

if cuisine_col:
    # cuisines might be pipe/comma separated
    s = data_clean[cuisine_col].dropna().astype(str)
    # split by common separators
    import re
    split_cuisines = s.str.split(r',|\||;')
    exploded = split_cuisines.explode().str.strip().str.title()
    top10 = exploded.value_counts().head(20)
    display(top10.head(10))

    # Max and min number of cuisines a restaurant serves
    cuisine_counts = split_cuisines.apply(lambda x: len([i for i in x if str(i).strip()!='']) if isinstance(x,list) else 0)
    print("Max cuisines per restaurant:", cuisine_counts.max())
    print("Min cuisines per restaurant:", cuisine_counts.min())

    # Most served cuisine for each city
    if city_col:
        def most_common_in_city(df, city):
            csplit = df[df[city_col]==city][cuisine_col].dropna().astype(str).str.split(r',|\||;').explode().str.strip().str.title()
            if len(csplit):
                return csplit.value_counts().idxmax()
            return None
        top_cuisine_by_city = {city: most_common_in_city(data_clean, city) for city in data_clean[city_col].dropna().unique()}
        # display for top 10 cities
        list(top_cuisine_by_city.items())[:10]

# 3. Cost distribution across restaurants
cost_col = None
for c in data_clean.columns:
    if 'cost' in c.lower() or 'price' in c.lower():
        cost_col = c
        break
print("Detected cost column:", cost_col)
if cost_col:
    # Try to coerce to numeric (remove non-digits)
    data_clean['__cost_num__'] = pd.to_numeric(data_clean[cost_col].astype(str).str.replace(r'[^0-9\.]','', regex=True), errors='coerce')
    display(data_clean['__cost_num__'].describe())
    # histogram
    plt.figure(figsize=(8,4))
    data_clean['__cost_num__'].dropna().hist(bins=30)
    plt.title("Cost distribution (numeric)")
    plt.xlabel("Cost for two (approx)")
    plt.ylabel("Count")
    plt.show()

# 4. Ratings distribution among various factors
# Find rating columns (may include 'Aggregate rating' and 'Rating text' etc.)
rating_cols = [c for c in data_clean.columns if 'rating' in c.lower() or 'rate' in c.lower()]
print("Detected rating-like columns:", rating_cols)
for rc in rating_cols:
    try:
        vals = pd.to_numeric(data_clean[rc], errors='coerce')
        print(f"\nStats for {rc}:")
        display(vals.describe())
        plt.figure(figsize=(6,3))
        vals.dropna().hist(bins=20)
        plt.title(f"Distribution of {rc}")
        plt.show()
    except Exception as e:
        print("Could not plot", rc, e)

# 5. Factors that may affect ratings: quick correlation and group comparisons
# Prepare a working DF with numeric candidates
work = data_clean.copy()
num_cols = []
for c in work.columns:
    if work[c].dtype in [np.float64, np.int64] or c in ['__votes_num__','__cost_num__']:
        num_cols.append(c)
# Ensure rating column present for correlation (choose first numeric rating-like column)
rating_numeric = None
for rc in rating_cols:
    tmp = pd.to_numeric(work[rc], errors='coerce')
    if tmp.notnull().sum()>0:
        work['__rating__'] = tmp
        rating_numeric = '__rating__'
        break

print("Numeric columns considered:", num_cols)
print("Chosen numeric rating column:", rating_numeric)

if rating_numeric:
    corr = work[[c for c in num_cols if c in work.columns] + [rating_numeric]].corr()[rating_numeric].sort_values(ascending=False)
    display(corr)
    plt.figure(figsize=(6,6))
    plt.barh(corr.index, corr.values)
    plt.title("Correlation with rating (numeric features)")
    plt.show()


In [ ]:

# 6-7. Tableau guidance
display(Markdown("## Tableau Dashboarding Guidance"))
display(Markdown(textwrap.dedent(\"\"\"
Suggested dashboards and worksheets to create in Tableau (data fields referenced are approximate — adjust to your actual column names):

1. **Overview / Map**: Plot restaurants on a map (use city or lat/lon if available). Size by number of votes, color by average rating. Filters: country, city, cuisine, cost for two, delivery/table booking flags.
2. **Top Restaurants / 'Star' Detection**: Create a composite 'star score' that combines normalized rating, votes, delivery availability, and (optionally) number of cuisines or cost bracket. Show top N restaurants with bar chart and table.
3. **Franchise Reach**: Horizontal bar showing number of unique cities per brand (restaurant name) to find franchises with widest presence.
4. **Cuisines Analysis**: Treemap or bar chart of top cuisines overall and by city. Enable filter to select city and see top cuisines there.
5. **Delivery vs Ratings**: Boxplot or violin plot comparing ratings for restaurants that deliver vs those who don't. Add filters for cuisine or cost brackets.
6. **Operational KPIs**: Percentage of restaurants providing online delivery, table booking ratio, average votes per restaurant, etc. Visualize as KPI tiles or donut charts.

**Calculated fields to create in Tableau** (examples):
- `Normalized Rating` = (`Rating` - WINDOW_MIN(Rating))/(WINDOW_MAX(Rating)-WINDOW_MIN(Rating))
- `Star Score` = 0.6 * `Normalized Rating` + 0.2 * LOG(`Votes`+1) + 0.1 * `Has Online Delivery` + 0.1 * `Has Table Booking`

\"\"\")))


In [ ]:

# Recommendation Engine — Simple recipe (no modeling code here, just guidance)
display(Markdown("## Recommendation Engine — Suggested approach"))

display(Markdown(textwrap.dedent(\"\"\"
1. **Feature engineering**:
   - Numeric features: normalized rating, log(votes+1), cost bracket, number of cuisines.
   - Binary flags: provides_delivery, provides_table_booking, is_franchise, has_online_order.
   - Categorical: city, cuisines (one-hot or embedding), restaurant type.

2. **User-item signals** (if available): past orders, clicks, favorites. If not available, create implicit signals using popularity (votes, visits) and recency.

3. **Model options**:
   - Content-based: compute similarity between restaurants using cuisine vectors + numeric features (cosine similarity). Recommend nearest neighbors.
   - Collaborative / Hybrid: if user interaction data exists, use matrix factorization or LightFM / implicit libraries to combine content and collaborative signals.
   - Ranking: train a learning-to-rank model using features above to rank candidate restaurants for a user/city.

4. **Evaluation metrics**: Precision@K, Recall@K, NDCG, and offline A/B testing for live results.

5. **Deployment**: Precompute embeddings and nearest neighbors (using Annoy, Faiss) for fast online recommendations.
\"\"\")))


In [ ]:

# (This cell is informative — the notebook file has already been written by the creator script. 
# If you re-run or modify the notebook, save using the Jupyter UI or this command:)
# nbformat.write(nb, "restaurant_consolidator_project.ipynb")
print("Notebook ready. Use 'File -> Download' in the Jupyter UI to get the .ipynb, or use the provided download link.")
